In [ ]:
import os

if not os.path.exists('Output/'):
    raise Exception("Models not found, pls run 'Train' first.")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_gen = ImageDataGenerator(rescale=1./255)

test_gen_output = test_gen.flow_from_directory('data_set/test', target_size=(256, 256), 
                                            batch_size=536, class_mode='binary')

In [ ]:
x_test = []
y_test = []
for x, y in test_gen_output:
    x_test = x
    y_test = y
    break

# EvaluationVisualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

def show_confusion_matrix(conf_matrix):
    ConfusionMatrixDisplay(conf_matrix, display_labels=['COVID','NORMAL']).plot()

In [ ]:
def show_model_chart(label, accuracy, precision, recall, f1):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_title(label + " Summary")
    bar_values = map(lambda x: x * 100, [accuracy, precision, recall, f1])
    ax.bar(['Accuracy', 'Precision', 'Recall', 'F1 Score'], list(bar_values), 0.35)
    ax.set_xlabel('Model evaluation metrics', fontweight ='bold', fontsize = 15)
    ax.set_ylabel('Percentage', fontweight ='bold', fontsize = 15) 
    plt.show()

# ModelEvaluateFunction

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def evaluate(label, model):
    print('\n*****************',label, '*****************\n')
    model.summary()
    
    result = model.predict(x_test)
    
    y_pred = []
    for i in result.flatten():
        if i >= 0.5:
            y_pred.append(1) 
        else:
            y_pred.append(0) 
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    print('TP :', tp, ', TN :', tn,', FP :', fp,', FN :', fn)
    show_confusion_matrix(conf_matrix)
    
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy :', accuracy)
    
    precision = precision_score(y_test, y_pred, average = "macro")
    print('Precision :', precision)
    
    recall = recall_score(y_test, y_pred, average = "macro")
    print('Recall :', recall)
    
    f1 = f1_score(y_test, y_pred, average = "macro")
    print('F1 score :', f1)
    
    show_model_chart(label, accuracy, precision, recall, f1)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
evaluate('Scratch model', load_model('Output/ScratchModel.h5'))

In [ ]:
evaluate('Resnet model', load_model('Output/ResnetModel.h5'))

In [ ]:
evaluate('Xception model', load_model('Output/XceptionModel.h5'))